In [1]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio openai-whisper spacy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 95.0 MB/s eta 0:00:00


In [3]:
import whisper
import gradio as gr
import spacy

In [4]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 123MiB/s]


In [5]:
nlp = spacy.load("en_core_web_sm")

In [31]:
def extract_instructions(sentence):
    # parsing
    doc = nlp(sentence.lower())

    move_distance = None
    up_distance = None
    move_keywords = {"move", "go", "proceed", "advance", "forward", "ahead"}
    up_keywords = {"fly", "ascend", "up", "rise"}

    for token in doc:
      #intent
        if token.lemma_ in move_keywords and move_distance is None:
            for ent in doc.ents:
                if ent.label_ == "CARDINAL" and ent.start > token.i:
                    move_distance = float(ent.text)
                    break

        if token.lemma_ in up_keywords and up_distance is None:
            for ent in doc.ents:
                if ent.label_ == "CARDINAL" and ent.start > token.i:
                    up_distance = float(ent.text)
                    break

    # default
    move_distance = move_distance or 5.0
    up_distance = up_distance or 3.0

    return move_distance, up_distance

In [32]:
def transcribe_and_process(audio_path):
    result = model.transcribe(audio_path)
    transcription = result['text']

    move_distance, up_distance = extract_instructions(transcription)

    if move_distance and up_distance:
        response = f"Moving forward by {move_distance} meters and flying up by {up_distance} meters."
    elif move_distance:
        response = f"Moving forward by {move_distance} meters."
    elif up_distance:
        response = f"Flying up by {up_distance} meters."
    else:
        response = "No movement instruction detected."
    return transcription, response

In [33]:
iface = gr.Interface(
    fn=transcribe_and_process,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Textbox(label="Transcription"), gr.Textbox(label="Command")],
    title="Robot Control Command Transcriber",
)

In [34]:
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1de17d51a0ae49875e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
